In [ ]:
from Bio import Entrez, SeqIO
from Bio.SeqUtils import gc_fraction
from Bio.Seq import Seq
from itertools import product
from collections import Counter
import pandas as pd
import os
from pathlib import Path
from BCBio import GFF
import re
import itertools



In [2]:
input_folder = "../data/classified"
output_folder = "../data/feature_extraction"

# Map each input file to its matching genome accession
genome_mapping = {
    "Brandao_MCCM_full_raw_counts_tpm_filtered_classified.tsv": "NC_010326",
    "Finstrlova_Newman_full_raw_counts_tpm_filtered_classified.tsv": "NC_005880",
    "Guegler_T4_minusToxIN_full_raw_counts_tpm_filtered_classified.tsv": "NC_000866",
    "Guegler_T7_plusToxIN_full_raw_counts_tpm_filtered_classified.tsv": "NC_001604",
    "Lood_full_raw_counts_tpm_filtered_classified.tsv": "MK797984.1",
    "Sprenger_VC_WT_VP882_delta_cpdS_full_raw_counts_tpm_filtered_classified.tsv": "NC_009016.1",
    "Yang_full_raw_counts_tpm_filtered_classified.tsv": "NC_021316",
}

# Make sure output folder exists
os.makedirs(output_folder, exist_ok=True)

# ---- FUNCTION TO GET PROTEIN + DNA SEQUENCE ----
def get_sequences_from_geneid(genome_accession, geneid):
    try:
        handle = Entrez.efetch(db="nucleotide", id=genome_accession, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()
    except Exception as e:
        print(f"⚠ Failed to fetch {genome_accession}: {e}")
        return ("ERROR_FETCH", "ERROR_FETCH")

    tag = geneid.replace("gene-", "").strip()
    short_tag = tag.split("_")[-1]

    for feature in record.features: 
        if feature.type == "CDS":
            locus_tag = feature.qualifiers.get("locus_tag", [""])[0]
            gene = feature.qualifiers.get("gene", [""])[0]
            product = feature.qualifiers.get("product", [""])[0]

            if tag in [locus_tag, gene, product] or short_tag in [locus_tag, gene, product]:
                protein = feature.qualifiers.get("translation", ["TRANSLATION_NOT_FOUND"])[0]
                dna_seq = feature.location.extract(record.seq)
                return (protein, str(dna_seq))

    return ("NOT_FOUND", "NOT_FOUND")

# ---- MAIN PROCESSING ----
for file_name in os.listdir(input_folder):
    if not file_name.endswith(".tsv"):
        continue

    print(f"🔍 Processing: {file_name}")
    file_path = os.path.join(input_folder, file_name)
    df = pd.read_csv(file_path, sep="\t")

    genome_acc = genome_mapping.get(file_name)
    if not genome_acc:
        print(f"⚠ No genome accession mapped for {file_name}")
        continue

    protein_seqs = []
    dna_seqs = []

    for geneid in df["Geneid"]:
        protein, dna = get_sequences_from_geneid(genome_acc, geneid)
        protein_seqs.append(protein)
        dna_seqs.append(dna)

    df["ProteinSequence"] = protein_seqs
    df["DNASequence"] = dna_seqs

    out_path = os.path.join(output_folder, file_name)
    df.to_csv(out_path, sep="\t", index=False)
    print(f"✅ Saved to: {out_path}")

🔍 Processing: Brandao_MCCM_full_raw_counts_tpm_filtered_classified.tsv


/home/lelo3/anaconda3/envs/vividVirions/lib/python3.13/site-packages/Bio/Entrez/__init__.py:734: UserWarning: 
            Email address is not specified.

            To make use of NCBI's E-utilities, NCBI requires you to specify your
            email address with each request.  As an example, if your email address
            is A.N.Other@example.com, you can specify it as follows:
               from Bio import Entrez
               Entrez.email = 'A.N.Other@example.com'
            In case of excessive usage of the E-utilities, NCBI will attempt to contact
            a user at the email address provided before blocking access to the
            E-utilities.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Folder containing your result files
results_folder = "../data/feature_extraction"

# Keywords to look for in failed extractions
failure_keywords = ["NOT_FOUND", "ERROR_FETCH", "TRANSLATION_NOT_FOUND"]

# Storage for summary
summary = {}

# Loop through all TSV files
for file_name in os.listdir(results_folder):
    if not file_name.endswith(".tsv"):
        continue

    file_path = os.path.join(results_folder, file_name)
    df = pd.read_csv(file_path, sep="\t")

    failed_rows = df[
        df["ProteinSequence"].isin(failure_keywords) |
        df["DNASequence"].isin(failure_keywords)
    ]

    if not failed_rows.empty:
        summary[file_name] = failed_rows[["Geneid", "ProteinSequence", "DNASequence"]]

# Report the results
if summary:
    print("❌ Failed sequence extractions found in the following files:\n")
    for fname, failures in summary.items():
        print(f"📄 {fname} — {len(failures)} failures")
        print(failures.to_string(index=False))
        print("-" * 60)
else:
    print("✅ All sequence extractions succeeded. No issues found.")

❌ Failed sequence extractions found in the following files:

📄 Finstrlova_Newman_full_raw_counts_tpm_filtered_classified.tsv — 4 failures
               Geneid ProteinSequence DNASequence
gene-CPT_phageK_gt004       NOT_FOUND   NOT_FOUND
gene-CPT_phageK_gt002       NOT_FOUND   NOT_FOUND
gene-CPT_phageK_gt003       NOT_FOUND   NOT_FOUND
gene-CPT_phageK_gt001       NOT_FOUND   NOT_FOUND
------------------------------------------------------------
📄 Guegler_T4_minusToxIN_full_raw_counts_tpm_filtered_classified.tsv — 10 failures
     Geneid ProteinSequence DNASequence
gene-T4t006       NOT_FOUND   NOT_FOUND
gene-T4t003       NOT_FOUND   NOT_FOUND
gene-T4t008       NOT_FOUND   NOT_FOUND
gene-T4t007       NOT_FOUND   NOT_FOUND
gene-T4s002       NOT_FOUND   NOT_FOUND
gene-T4t001       NOT_FOUND   NOT_FOUND
gene-T4s001       NOT_FOUND   NOT_FOUND
gene-T4t002       NOT_FOUND   NOT_FOUND
gene-T4t004       NOT_FOUND   NOT_FOUND
gene-T4t005       NOT_FOUND   NOT_FOUND
-----------------------------

# User-story 11: Sequence based features
@LuiseJedlitschka  
@milli2908  
@elivic734    
    
## Potential features:
- **CG-count**: Despite being a simple metric, GC presents a huge variation across genomes (ranging from approximately 20% to 70% ). GC content is reasonably constant within a given genome, and was already found to be correlated with several universal factors of microbial lifestyles such as temperature, niche complexity and aerobiosis [Genomic Signature](https://www.sciencedirect.com/topics/biochemistry-genetics-and-molecular-biology/genomic-signature).
- **sequence-length**
- **K-Mer-frequency**: e.g. k3 -> codon composition
- **nucleotide composition** (Base percentages and purin/pyrimidin percentages)
- **CpG bias**: relevant for interaction with host, methylation, epigenetic regulation, 
 Interpretation:

    CpG < 1: CpG is underrepresented

    CpG ≈ 1: expected frequency, no bias

    CpG > 1: CpG is overrepresented
- **motives**:

    "TATA_box": "TATAAA",       # Promotor

    "CAAT_box": "GGCCAATCT",    # Enhancer

    "PolyA_signal": "AATAAA",   # Signal for transcriptation
- **relative position in genome**
    



## User-story 13: integration of external insights
@LuiseJedlitschka  
@milli2908  
@elivic734    
  
[Genomic features of E. coli O177 phages](https://www.nature.com/articles/s41598-023-48788-w/tables/1): sequence length, GC-content

*comparison*: also used as features in our model

[DeepPL: A deep-learning-based tool for the prediction of bacteriophage lifecycle](https://pmc.ncbi.nlm.nih.gov/articles/PMC11521287/): K-mer 6 was selected in this study based on its best performance, as reported by DNABERT

*comparison*: the sliding window approach is used to detect the Tricolons, as our input is not a whole genome but already specific geneso used in our model

[Digital phagograms: predicting phage infectivity through a multilayer machine learning approach](https://www.sciencedirect.com/science/article/pii/S1879625721001620): 
Nucleotide composition, Codon composition, Codon usage bias (Codon usage bias refers to the differences in the frequency of usage for a certain base triplet for a given amino acid among organisms), GC content, CpG bias, k-mer frequeR spacers, Embeddings, pVOG hits  Coding DNA sequences	(Auxiliary metabolic genes, Shared tRNAs, defense system genes, Physicochemical properties)

*comparison*: only a few of these (nucleotide composition, Codon composition, GC content, k-mer frequencies, CpG bias) are used in our model as some are redundant with the structural features relying (pVOG hits, Coding DNA sequences, Physicochemical properties) on the protein sequence, some are only necessary when the and the host-genome (CRISPR spacers, Shared tRNAs, defense system genes) input is a whole genome, not just sequences like in our case, and some are just too specific considering our main focus is on the protein-sequences-features (Codon usage bias)

[PhageAI - Bacteriophage Life Cycle Recognition with Machine Learning and Natural Language Processing](https://www.biorxiv.org/content/10.1101/2020.07.11.198606v1.full): sliding window approach using constant k = 6 and the Word2Vec algorithm with the Skip-gram model, nominal features were empirically chosen by feature selection algorithm called Feature ranking with recursive feature elimination and cross-validated selection of the best number of features using Support Vector Machine 

*comparison*: sliding window approach is used to detect the Tricolons, as our input is not a whole genome but already specific genes

[Rapid discovery of novel prophages using biological feature engineering and machine learning](https://academic.oup.com/nargab/article/3/1/lqaa109/6066536?login=true): gc_content (also in specific positions of the codons), CAI (codon adaptation index, measures the codon usage bias), BP:percent (for all bases, purin, pyrimidin)

*comparison*: the most important (gc_content and BP:percentage) are in our model, the others are very detailed and unnecessary as our features focus on protein sequenceson: also used in our model
                                                                                                        


In [4]:
# Load feature_extraction table
input_path = Path('../data/feature_extraction')
output_path = Path('../data/dna_feature_table')
gff_folder = Path("../data/features")
output_path.mkdir(exist_ok=True)

# Computing k-mers + Counting k-mers for each sequence
k = 3

# Creates every possible k-mer of the length 3
alphabet = ["A", "T", "G", "C"]
all_kmers = ["".join(p) for p in product(alphabet, repeat=k)]
top_k_features = 10

# Helper function to count k-mers 
def count_kmers(seq, k):
    seq = seq.upper()
    counts = Counter(seq[i:i+k] for i in range(len(seq) - k + 1) if set(seq[i:i+k]).issubset(alphabet))
    return [counts.get(kmer, 0) for kmer in all_kmers] # returns a list with amount of each k-mer of a sequence
    
for file in input_path.glob("*.tsv"):
    df = pd.read_csv(file, sep="\t")
    
    # Only keep relevant columns
    df_features = df[["Geneid", "DNASequence", "classification"]].copy()

    # Extracting Gene positions from GFF
    base_name = file.name.split("_")[0].lower()
    gff_match = None
    for gff_file in gff_folder.glob("*.gff3"):
        if gff_file.name.lower().startswith(base_name):
            gff_match = gff_file
            break
    if gff_match:
        gene_positions = {}
        # Parse the GFF file to extract gene positions
        with open(gff_match) as in_handle:
            for rec in GFF.parse(in_handle):
                genome_length = len(rec)
                for feature in rec.features:
                    if feature.type == "gene":
                        gene_id = feature.id
                        start = int(feature.location.start)
                        end = int(feature.location.end)
                        midpoint = (start + end) // 2
                        gene_positions[gene_id] = (start, end, midpoint)
        # Mapping the gene positions to the feature table                
        df_features["Gene_Position_Start"] = df_features["Geneid"].map(lambda gid: gene_positions.get(gid, (None, None, None))[0])
        df_features["Gene_Position_End"] = df_features["Geneid"].map(lambda gid: gene_positions.get(gid, (None, None, None))[1])
        df_features["Gene_Position_Midpoint"] = df_features["Geneid"].map(lambda gid: gene_positions.get(gid, (None, None, None))[2])
        # Calculate the relative position of each gene ( midpoint / genome length)
        df_features["Gene_Position_Relative"] = df_features["Gene_Position_Midpoint"].apply(lambda x: x / genome_length if x is not None else None)
        # Remove Start, End and Midpoint columns
        df_features.drop(["Gene_Position_Start", "Gene_Position_End", "Gene_Position_Midpoint"], axis=1, inplace=True)
    else:
        print(f"No GFF found for {file.name}, skipping position features.")


    # Computing GC-content for each sequence
    df_features["GC_Content"] = df_features["DNASequence"].apply(lambda seq: gc_fraction(Seq(seq)))

      
    # Computing sequence length for each DNA sequence                                                      
    df_features["Seq_length"] = df_features["DNASequence"].apply(lambda seq: len(seq) if isinstance(seq, str) else 0) 

    # Function to count bases
    def count_bases(seq):
        seq = seq.upper()
        return {
            "A_Content": seq.count("A"),
            "T_Content": seq.count("T"),
            "G_Content": seq.count("G"),
            "C_Content": seq.count("C")
        }
    
    motives = {
        "TATA_box": "TATAAA",       # Promotor
        "CAAT_box": "GGCCAATCT",    # Enhancer
        "PolyA_signal": "AATAAA",   # Signal for transcriptation
    }

    def count_motives(sequence, motives):
        sequence = sequence.upper()
        result = {}
        for name, pattern in motives.items():
            result[f"motive_{name}"] = len(re.findall(pattern, sequence))
        return result
    
    # Counting bases A,G,C and T
    base_counts = df_features["DNASequence"].apply(count_bases).apply(pd.Series)
    base_fractions = base_counts.div(df_features["Seq_length"], axis=0)
    df_features = pd.concat([df_features, base_fractions], axis=1)
    
    # Counting Purin (A + G) and Pyrimidin (C + T)
    df_features["Purin_Content"] = df_features["DNASequence"].apply(lambda seq: (seq.count("A") + seq.count("G")) / len(seq))
    df_features["Pyrimidin_Content"] = df_features["DNASequence"].apply(lambda seq: (seq.count("C") + seq.count("T")) / len(seq))
    
    # Computing CpG bias
    def calc_cpg_bias(row):
        # Annahme: row['DNASequence'], row['G-content'], row['C-content'], row['Seq_length'] existieren
        seq = row['DNASequence'].upper()
        c_count = seq.count("C")
        g_count = seq.count("G")
        seq_length = len(seq)
        cpg_count = sum(1 for i in range(seq_length-1) if seq[i:i+2] == "CG")
        expected = (c_count * g_count) / seq_length if seq_length > 0 else 0
        return (cpg_count / expected) if expected > 0 else 0

    df_features["CpG_bias"] = df_features.apply(calc_cpg_bias, axis=1)

    
    # Counting motives
    motive_features_rel = df_features.apply(
    lambda row: pd.Series({f"{k}_rel": v / row["Seq_length"] if row["Seq_length"] > 0 else 0
                           for k, v in count_motives(row["DNASequence"], motives).items()}),
    axis=1
    )
    df_features = pd.concat([df_features, motive_features_rel], axis=1)
    
    # k-mer counting
    kmer_counts = df_features["DNASequence"].apply(lambda seq: count_kmers(seq, k))
    kmer_df = pd.DataFrame(kmer_counts.tolist(), columns=[f'kmer_{kmer}' for kmer in all_kmers])

    # Normalize k-mer counts
    kmer_sums = kmer_df.sum(axis=1).replace(0, 1)
    kmer_df_norm = kmer_df.div(kmer_sums, axis=0)

    
    # Save as new TSV
    df_out = pd.concat([df_features, kmer_df_norm], axis=1)
    out_file = output_path / file.name
    df_out.to_csv(out_file, sep="\t", index=False)
    print(f"Features table saved as {output_path}")    





Features table saved as ../data/dna_feature_table
Features table saved as ../data/dna_feature_table
Features table saved as ../data/dna_feature_table
Features table saved as ../data/dna_feature_table
Features table saved as ../data/dna_feature_table
Features table saved as ../data/dna_feature_table
Features table saved as ../data/dna_feature_table


# User-story 11: Sequence based features
@dottting  
@davmar01

The following code computes feature vectors for each sequence using amino acid composition (AAC) and dipeptide (k=2) frequencies, capturing both global and local sequence properties. The feature matrices are save at "../features/protein_primary_k=1and2". 

In [18]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis

input_folder = "../data/feature_extraction"
output_folder = "../features/protein/protein_primary_k=1_k=2"
os.makedirs(output_folder, exist_ok=True)

# List of standard amino acids
AA_LIST = list("ACDEFGHIKLMNPQRSTVWY")

# Generate all possible 2-mer combinations (dipeptides)
DIPEPTIDES = ["".join(p) for p in product(AA_LIST, repeat=2)]


def extract_aac_kmer_features(seq):
    """Extract AAC, dipeptide (k=2), and key physicochemical features from a protein sequence."""
    seq = seq.upper().replace("*", "")
    total_len = len(seq)

    # AAC features
    aac_counts = Counter(seq)
    aac_features = {f"AAC_{aa}": aac_counts.get(aa, 0) / total_len for aa in AA_LIST}

    # Dipeptide (k=2) features
    kmer_counts = Counter(seq[i : i + 2] for i in range(len(seq) - 1))
    total_kmers = sum(kmer_counts.values()) or 1  # avoid division by zero
    kmer_features = {
        f"kmer_{dp}": kmer_counts.get(dp, 0) / total_kmers for dp in DIPEPTIDES
    }

    # Physicochemical features
    try:
        analysis = ProteinAnalysis(seq)
        pI = analysis.isoelectric_point()
        gravy = analysis.gravy()
        instability = analysis.instability_index()
    except Exception:
        pI, gravy, instability = None, None, None

    extra_features = {
        "IsoelectricPoint": pI,
        "GRAVY": gravy,
        "InstabilityIndex": instability,
        "seq_length": total_len,
    }

    return {**aac_features, **kmer_features, **extra_features}


# Process each .tsv file in the input directory
for filename in os.listdir(input_folder):
    if not filename.endswith(".tsv"):
        continue

    filepath = os.path.join(input_folder, filename)

    try:
        df = pd.read_csv(filepath, sep="\t")
        df.columns = df.columns.str.strip().str.lower()
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        continue

    # Check for required columns
    required = {"geneid", "classification", "proteinsequence"}
    if not required.issubset(df.columns):
        print(f"Skipping file {filename} – required columns missing.")
        print(f"Found columns: {df.columns.tolist()}")
        continue

    # Extract features row-wise
    all_features = []
    for _, row in df.iterrows():
        try:
            feats = extract_aac_kmer_features(row["proteinsequence"])
            ordered_feats = {
                "Geneid": row["geneid"],
                "classification": row["classification"],
                **feats,
            }
            all_features.append(ordered_feats)
        except Exception as e:
            print(f"Error in row with gene ID {row.get('geneid', 'unknown')}: {e}")

    # Save the resulting feature matrix
    if all_features:
        output_df = pd.DataFrame(all_features)
        output_filename = filename.replace(".tsv", "_features.tsv")
        output_path = os.path.join(output_folder, output_filename)
        output_df.to_csv(output_path, sep="\t", index=False)
        print(f"Saved: {output_path}")
    else:
        print(f"No valid entries in {filename}, nothing saved.")

Saved: ../features/protein/protein_primary_k=1_k=2\Brandao_MCCM_full_raw_counts_tpm_filtered_classified_features.tsv
Saved: ../features/protein/protein_primary_k=1_k=2\Finstrlova_Newman_full_raw_counts_tpm_filtered_classified_features.tsv
Saved: ../features/protein/protein_primary_k=1_k=2\Guegler_T4_minusToxIN_full_raw_counts_tpm_filtered_classified_features.tsv
Saved: ../features/protein/protein_primary_k=1_k=2\Guegler_T7_plusToxIN_full_raw_counts_tpm_filtered_classified_features.tsv
Saved: ../features/protein/protein_primary_k=1_k=2\Lood_full_raw_counts_tpm_filtered_classified_features.tsv
Saved: ../features/protein/protein_primary_k=1_k=2\Sprenger_VC_WT_VP882_delta_cpdS_full_raw_counts_tpm_filtered_classified_features.tsv
Saved: ../features/protein/protein_primary_k=1_k=2\Yang_full_raw_counts_tpm_filtered_classified_features.tsv


### Tripedtide features (most commonly used)

In [19]:
output_folder = "../features/protein/protein_primary_table_k=3"
os.makedirs(output_folder, exist_ok=True)

TRIPEPTIDES = ["".join(p) for p in product(AA_LIST, repeat=3)]

def extract_k3_features(seq):
    """Extract tripeptide (k=3) frequency features + top 3 physicochemical features from a protein sequence."""
    seq = seq.upper().replace("*", "")
    
    # Tripeptide features
    tri_counts = Counter(seq[i:i+3] for i in range(len(seq) - 2))
    total_tri = sum(tri_counts.values()) or 1
    tri_features = {
        f"kmer3_{tp}": tri_counts.get(tp, 0) / total_tri for tp in TRIPEPTIDES
    }

    # Physicochemical features
    try:
        analysis = ProteinAnalysis(seq)
        pI = analysis.isoelectric_point()
        gravy = analysis.gravy()
        instability = analysis.instability_index()
    except Exception:
        pI, gravy, instability = None, None, None

    extra_features = {
        "IsoelectricPoint": pI,
        "GRAVY": gravy,
        "InstabilityIndex": instability,
        "seq_length": len(seq),
    }

    return {**tri_features, **extra_features}

# Process each .tsv file in the input directory
for filename in os.listdir(input_folder):
    if not filename.endswith(".tsv"):
        continue

    filepath = os.path.join(input_folder, filename)

    try:
        df = pd.read_csv(filepath, sep="\t")
        df.columns = df.columns.str.strip().str.lower()
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        continue

    # Check for required columns
    required = {"geneid", "classification", "proteinsequence"}
    if not required.issubset(df.columns):
        print(f"Skipping file {filename} – required columns missing.")
        print(f"Found columns: {df.columns.tolist()}")
        continue

    # Extract features row-wise
    all_features = []
    for _, row in df.iterrows():
        try:
            feats = extract_k3_features(row["proteinsequence"])
            ordered_feats = {
                "Geneid": row["geneid"],
                "classification": row["classification"],
                **feats,
            }
            all_features.append(ordered_feats)
        except Exception as e:
            print(f"Error in row with gene ID {row.get('geneid', 'unknown')}: {e}")

    # Save the resulting feature matrix
    if all_features:
        output_df = pd.DataFrame(all_features)
        output_filename = filename.replace(".tsv", "_k3_physchem.tsv")
        output_path = os.path.join(output_folder, output_filename)
        output_df.to_csv(output_path, sep="\t", index=False)
        print(f"Saved k=3+physicochemical features to: {output_path}")
    else:
        print(f"No valid entries in {filename}, nothing saved.")

Saved k=3+physicochemical features to: ../features/protein/protein_primary_table_k=3\Brandao_MCCM_full_raw_counts_tpm_filtered_classified_k3_physchem.tsv
Saved k=3+physicochemical features to: ../features/protein/protein_primary_table_k=3\Finstrlova_Newman_full_raw_counts_tpm_filtered_classified_k3_physchem.tsv
Saved k=3+physicochemical features to: ../features/protein/protein_primary_table_k=3\Guegler_T4_minusToxIN_full_raw_counts_tpm_filtered_classified_k3_physchem.tsv
Saved k=3+physicochemical features to: ../features/protein/protein_primary_table_k=3\Guegler_T7_plusToxIN_full_raw_counts_tpm_filtered_classified_k3_physchem.tsv
Saved k=3+physicochemical features to: ../features/protein/protein_primary_table_k=3\Lood_full_raw_counts_tpm_filtered_classified_k3_physchem.tsv
Saved k=3+physicochemical features to: ../features/protein/protein_primary_table_k=3\Sprenger_VC_WT_VP882_delta_cpdS_full_raw_counts_tpm_filtered_classified_k3_physchem.tsv
Saved k=3+physicochemical features to: ../f

### Secondary Structure Features

- Predict structures with ESM3  (see `data/protein_structures/pdb`)

- Secondary-structure related
  - Secondary-structure content (α-helices, β-sheets, coils)  
  - Solvent accessibility  
  - Secondary-structure transitions  

- ESM3 embeddings (TODO)